<a href="https://colab.research.google.com/github/Dixit6054/stockfetch/blob/master/trading_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing data

In [ ]:
# This cell imports all the necessary modules.
!pip install -U kaleido
import kaleido
import requests
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import time
import pytz



In [ ]:
import requests
import pandas as pd

def fetch_kraken_ohlc(symbol: str = "XBTUSD", interval: int = 60, limit: int = 100):
    url = "https://api.kraken.com/0/public/OHLC"
    params = {
        "pair": symbol,
        "interval": interval,
    }
    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Error fetching data from Kraken API: {response.json()}")

    data = response.json()
    if 'error' in data and data['error']:
        raise Exception(f"Kraken API Error: {data['error']}")

    # Extract OHLC data
    pair_data = list(data["result"].values())[0]
    # Updated columns to include the extra column from the API response
    columns = ["Time", "Open", "High", "Low", "Close", "Volume", "Vwep", "Count"]
    df = pd.DataFrame(pair_data, columns=columns)

    # Convert to appropriate types
    df["Time"] = pd.to_datetime(df["Time"], unit='s')
    numeric_cols = ["Open", "High", "Low", "Close", "Volume", "Vwep"] #Include Vwep in numeric columns
    df[numeric_cols] = df[numeric_cols].astype(float)
    df.set_index("Time", inplace=True)

    #Calculate Close Time - shifted by the interval
    df['Close Time'] = df.index + pd.Timedelta(seconds=interval)

    return df

# Example usage
df = fetch_kraken_ohlc()
print(df.head())

                        Open     High      Low    Close   Volume        Vwep  \
Time                                                                           
2024-11-17 22:00:00  89147.8  89775.5  88728.6  89488.0  89314.7  377.105235   
2024-11-17 23:00:00  89530.4  89850.0  89374.3  89850.0  89666.0   16.954050   
2024-11-18 00:00:00  89850.0  90096.1  89700.0  89920.1  89960.5   60.228627   
2024-11-18 01:00:00  89942.6  90888.8  89913.6  90737.0  90479.6  126.554994   
2024-11-18 02:00:00  90737.1  90831.4  90417.2  90484.3  90577.5   48.874748   

                     Count          Close Time  
Time                                            
2024-11-17 22:00:00   2373 2024-11-17 22:01:00  
2024-11-17 23:00:00   1151 2024-11-17 23:01:00  
2024-11-18 00:00:00   1312 2024-11-18 00:01:00  
2024-11-18 01:00:00   1614 2024-11-18 01:01:00  
2024-11-18 02:00:00   1142 2024-11-18 02:01:00  


In [ ]:
#import from binance does not work from canada /US
# this function does not do anything, just for reference

import requests
import pandas as pd

def fetch_binance_klines(symbol: str, interval: str, limit: int = 100):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(url, params=params)
    data = response.json()

    df = pd.DataFrame(data, columns=[
        "Open Time", "Open", "High", "Low", "Close", "Volume",
        "Close Time", "Quote Asset Volume", "Number of Trades",
        "Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"
    ])

    df["Open Time"] = pd.to_datetime(df["Open Time"], unit='ms', utc=True)
    df["Close Time"] = pd.to_datetime(df["Close Time"], unit='ms', utc=True)
    numeric_cols = ["Open", "High", "Low", "Close", "Volume"]
    df[numeric_cols] = df[numeric_cols].astype(float)
    df.set_index("Open Time", inplace=True)

    return df[["Open", "High", "Low", "Close", "Volume", "Close Time"]]

# Example usage
df = fetch_binance_klines("BTCUSDT", "1h", limit=100)
print(df.head())

Empty DataFrame
Columns: [Open, High, Low, Close, Volume, Close Time]
Index: []


# Plot Candles

In [ ]:
def plot_candles(h1_df, d1_df):
    """
    Plot 2 columns:
    - Column 1: last 20 1H candles (including current candle forming)
    - Column 2: last 5 1D candles (including current candle forming)

    Show time in EST, no scroller, separate Y scales.
    Legend at top, axis labels visible.
    Annotate the current candle with time until it closes.
    """
    # Extract the subsets
    h1_data = h1_df.tail(20)
    d1_data = d1_df.tail(5)

    # Convert index and close times to EST
    est_tz = pytz.timezone("America/New_York")
    h1_data.index = h1_data.index.tz_localize('UTC').tz_convert(est_tz)
    #h1_data["Close Time"] = h1_data["Close Time"].dt.tz_localize('UTC').tz_convert(est_tz)
    h1_data["Close Time"] = h1_data["Close Time"].dt.tz_localize('UTC').tz_convert(est_tz)

    d1_data.index = d1_data.index.tz_localize('UTC').tz_convert(est_tz)
    d1_data["Close Time"] = d1_data["Close Time"].dt.tz_localize('UTC').tz_convert(est_tz)

    fig = make_subplots(rows=1, cols=2, shared_yaxes=False, horizontal_spacing=0.05)

    # 1H Candles
    fig.add_trace(
        go.Candlestick(
            x=h1_data.index,
            open=h1_data["Open"], high=h1_data["High"],
            low=h1_data["Low"], close=h1_data["Close"],
            name="1H"
        ),
        row=1, col=1
    )

    # 1D Candles
    fig.add_trace(
        go.Candlestick(
            x=d1_data.index,
            open=d1_data["Open"], high=d1_data["High"],
            low=d1_data["Low"], close=d1_data["Close"],
            name="1D"
        ),
        row=1, col=2
    )

    # Layout adjustments
    fig.update_layout(
        template="plotly_dark",
        margin=dict(l=40, r=40, t=40, b=40),
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.1,
            xanchor="left",
            x=0
        )
    )

    # Disable range slider
    fig.update_xaxes(rangeslider_visible=False)

    # Axes customization
    fig.update_xaxes(
        showgrid=False, zeroline=False, showline=True,
        showticklabels=True, linecolor='white',
        title_text="Time (EST)"
    )
    fig.update_yaxes(
        showgrid=True, gridcolor='gray', zeroline=False, showline=True,
        showticklabels=True, linecolor='white',
        title_text="Price"
    )

    # Annotate the current candle with time to close for each chart
    now_est = datetime.now(est_tz)

    # For 1H
    h1_data.iloc[-1]
    h1_last_candle = h1_data.iloc[-1]
    h1_close_time = h1_last_candle["Close Time"]
    h1_time_to_close = h1_close_time - now_est
    # Format time remaining as mm:ss (or hh:mm:ss if > 1h)
    h1_minutes, h1_seconds = divmod(int(h1_time_to_close.total_seconds()), 60)
    h1_hours, h1_minutes = divmod(h1_minutes, 60)
    if h1_hours > 0:
        h1_time_str = f"{h1_hours}h {h1_minutes}m {h1_seconds}s"
    else:
        h1_time_str = f"{h1_minutes}m {h1_seconds}s"
    fig.add_annotation(
        x=h1_data.index[-1],
        y=h1_data["High"].iloc[-1] * 1.01,  # slightly above candle
        text=f"Time to close: {h1_time_str}",
        showarrow=False,
        row=1, col=1,
        font=dict(color='white')
    )

    # For 1D
    d1_last_candle = d1_data.iloc[-1]
    d1_close_time = d1_last_candle["Close Time"]
    d1_time_to_close = d1_close_time - now_est
    d1_minutes, d1_seconds = divmod(int(d1_time_to_close.total_seconds()), 60)
    d1_hours, d1_minutes = divmod(d1_minutes, 60)
    d1_days, d1_hours = divmod(d1_hours, 24)
    # Format day/hour/min/sec
    if d1_days > 0:
        d1_time_str = f"{d1_days}d {d1_hours}h {d1_minutes}m {d1_seconds}s"
    elif d1_hours > 0:
        d1_time_str = f"{d1_hours}h {d1_minutes}m {d1_seconds}s"
    else:
        d1_time_str = f"{d1_minutes}m {d1_seconds}s"
    fig.add_annotation(
        x=d1_data.index[-1],
        y=d1_data["High"].iloc[-1] * 1.01,
        text=f"Time to close: {d1_time_str}",
        showarrow=False,
        row=1, col=2,
        font=dict(color='white')
    )

    return fig


In [ ]:
# Main Fetch to run for plots
def run_fetch_and_plot(symbol="BTCUSDT"):
    h1_df = fetch_kraken_ohlc(symbol, 60, limit=100)
    d1_df = fetch_kraken_ohlc(symbol, 60*24, limit=100)
    fig = plot_candles(h1_df, d1_df)
    fig.show()
    return fig

temp =run_fetch_and_plot("BTCUSDT")



<ipython-input-5-e39cbabad400>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e39cbabad400>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Export fig to Telegram Chat

In [ ]:
!pip install -U kaleido # Install or update kaleido
import kaleido          # Make sure kaleido is imported
import plotly.io as pio # Import the submodule used by Plotly for IO

# Test the run_fetch_and_plot function for BTC/USDT
fig = run_fetch_and_plot("BTCUSDT")
#fig.write_image("chart.png")

# Explicitly set the engine to 'kaleido'
pio.write_image(fig, "chart1.png", engine="kaleido")



<ipython-input-5-e39cbabad400>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e39cbabad400>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Exporting the Figure and Sending to Telegram

# Now send 'chart1.png' to Telegram.
# You need a Telegram bot token and chat_id.

TELEGRAM_BOT_TOKEN = "6752612939:AAHG52PLf3c8XK3Wc-t1ohszKeu8ILx4ISg"
CHAT_ID = "-1001958631566"

import requests

url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendPhoto"
files = {'photo': open('chart1.png', 'rb')}
data = {'chat_id' : CHAT_ID, 'caption': 'Latest 1H & 1D Candlesticks'}
r = requests.post(url, files=files, data=data)
print(r.json())


{'ok': True, 'result': {'message_id': 218, 'sender_chat': {'id': -1001958631566, 'title': 'GBPUSD', 'type': 'channel'}, 'chat': {'id': -1001958631566, 'title': 'GBPUSD', 'type': 'channel'}, 'date': 1734473252, 'photo': [{'file_id': 'AgACAgEAAx0EdL5YjgAD2mdh9iRvVe6s86rfdOzZuFvNJqerAAKFrjEbAyoRR20_gyBvKNKcAQADAgADcwADNgQ', 'file_unique_id': 'AQADha4xGwMqEUd4', 'file_size': 998, 'width': 90, 'height': 64}, {'file_id': 'AgACAgEAAx0EdL5YjgAD2mdh9iRvVe6s86rfdOzZuFvNJqerAAKFrjEbAyoRR20_gyBvKNKcAQADAgADbQADNgQ', 'file_unique_id': 'AQADha4xGwMqEUdy', 'file_size': 10802, 'width': 320, 'height': 229}, {'file_id': 'AgACAgEAAx0EdL5YjgAD2mdh9iRvVe6s86rfdOzZuFvNJqerAAKFrjEbAyoRR20_gyBvKNKcAQADAgADeAADNgQ', 'file_unique_id': 'AQADha4xGwMqEUd9', 'file_size': 31265, 'width': 700, 'height': 500}], 'caption': 'Latest 1H & 1D Candlesticks'}}
